In [1]:
def mod_mul_standard(a, b, p):
    result = (a * b) % p
    if result > (p>>1):
        result -= p
    return result

def mod_add(a, b, p):
    result = (a + b) 
    if result > (p>>1):
        result -= p
    if result < -(p>>1):
        result += p
    return result

def mod_sub(a, b, p):
    result = (a - b) 
    if result > (p>>1):
        result -= p
    if result < -(p>>1):
        result += p
    return result


def check_arrays(a,b,p):
    if a == b:
        print("exactly equal")
        return True
    print("not exactly equal")

    if len(a) != len(b):
        print("lengths not equal")
        return False


    for i in range(len(a)):
        if (a[i] - b[i]) % p != 0:
            print("element", i, "not equal even in modulus class")
            return False
    print("But equal in modulus class")
    return True

    



In [2]:
def Mu(b,q):
    mu = 2*int(((b << 31)/(2*q))+0.5)
    return mu


def Barrett_mul(a, b, p, mu_b):
    # Perform the multiplication
    z = (a * b) % (1 << 32)
    if z >= (1 << 31):
        z -= (1 << 32)  # Convert to signed 32-bit integer

    # Perform the multiplication for mu_b and get the high 32 bits
    temp = (a * (mu_b << 1)) % (1 << 64)
    t_high = (temp >> 32) % (1 << 32)
    if t_high >= (1 << 31):
        t_high -= (1 << 32)  # Convert to signed 32-bit integer
    
    result = z - t_high * p
    # Simulate 32-bit signed integer overflow for the result
    result %= (1 << 32)
    if result >= (1 << 31):
        result -= (1 << 32)  # Convert to signed 32-bit integer

    return result


def number_order(temp, p):
    maximum = max([abs(x) for x in temp])

    degree = 0
    while True:
        if degree*(p>>1) < maximum < (degree+1)*(p>>1):
            return degree
        degree += 1


In [3]:
def CT_4pts(P, t0, t1, t2, t3, omega_1, omega2, omega3, mu_omega_1, mu_omega2, mu_omega3):
    # First layer
    t2 = Barrett_mul(t2, omega_1, P, mu_omega_1)
    t3 = Barrett_mul(t3, omega_1, P, mu_omega_1)

    temp0 = mod_add(t0, t2, P)
    t2 = mod_sub(t0, t2, P)
    t0 = temp0

    temp1 = mod_add(t1, t3, P)
    t3 = mod_sub(t1, t3, P)
    t1 = temp1

    # Second layer
    t1 = Barrett_mul(t1, omega2, P, mu_omega2)
    t3 = Barrett_mul(t3, omega3, P, mu_omega3)

    temp0 = mod_add(t0, t1, P)
    t1 = mod_sub(t0, t1, P)
    t0 = temp0

    temp1 = mod_add(t2, t3, P)
    t3 = mod_sub(t2, t3, P)
    t2 = temp1

    return t0, t1, t2, t3

def GS_4pts(P, t0, t1, t2, t3, omega_1, omega2, omega3, mu_omega_1, mu_omega2, mu_omega3):
    # First layer
    temp0 = mod_add(t0, t1, P)
    t1 = mod_sub(t0, t1, P)
    t0 = temp0

    temp1 = mod_add(t2, t3, P)
    t3 = mod_sub(t2, t3, P)
    t2 = temp1

    t1 = Barrett_mul(t1, omega2, P, mu_omega2)
    t3 = Barrett_mul(t3, omega3, P, mu_omega3)

    # Second layer
    temp0 = mod_add(t0, t2, P)
    t2 = mod_sub(t0, t2, P)
    t0 = temp0

    temp1 = mod_add(t1, t3, P)
    t3 = mod_sub(t1, t3, P)
    t1 = temp1

    t2 = Barrett_mul(t2, omega_1, P, mu_omega_1)
    t3 = Barrett_mul(t3, omega_1, P, mu_omega_1)

    return t0, t1, t2, t3


In [4]:
from data import *

logn_top = 10
n = 1 << logn_top
temp = [0] *  (8 * n)

temp[0:n] = f[0:n]
temp[n:2*n] = g[0:n]

ptrf = 0
ptrg = n

In [5]:
hhn = n >> 2
start = 0
#temp[start:2*n:hhn] = [1,0,0,0,0,0,0,0]
print(temp[start:2*n:hhn])

[0, 0, 2, 1, 0, 1, 1, 3]


In [6]:
n = 1 << logn_top
hhn = n >> 2
power_index = 1


for j in range(0, 1):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1

In [7]:


print(number_order(temp, P[0]))

0


In [8]:
logn = logn_top - 2
n = 1 << logn
hhn = n >> 2
power_index = 4

for j in range(0, 4):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1

In [9]:
print(number_order(temp, P[0]))


0


In [10]:
logn = logn_top - 4
n = 1 << logn
hhn = n >> 2
power_index = 16
print(hhn,n)


for j in range(0, 16):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1



16 64


In [11]:
print(number_order(temp, P[0]))


0


In [12]:
logn = logn_top - 6
n = 1 << logn
hhn = n >> 2
power_index = 64

for j in range(0, 64):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1

In [13]:
logn = logn_top - 8
n = 1 << logn
hhn = n >> 2
power_index = 256

start = 0

for j in range(0, 256):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1



In [14]:


print(number_order(temp, P[0]))


0


In [15]:
# Bit-reverse index
def bitrev(i):
    rev = 0
    for j in range(10):
        rev = (rev << 1) | (i & 1)
        i >>= 1
    return rev


bitrev(1023)

1023

In [16]:
logn = logn_top - 8
n = 1 << logn
hhn = n >> 2
power_index = 511


for j in range(0, 256):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1


#print(ans)

In [17]:
logn = logn_top - 6
n = 1 << logn
hhn = n >> 2
power_index = 127

for j in range(0, 64):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [18]:
print(number_order(temp, P[0]))



2


In [19]:
logn = logn_top - 4
n = 1 << logn
hhn = n >> 2
power_index = 31

for j in range(0, 16):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [20]:
print(number_order(temp, P[0]))


2


In [21]:
logn = logn_top - 2
n = 1 << logn
hhn = n >> 2
power_index = 7

for j in range(0, 4):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [22]:
print(number_order(temp, P[0]))


2


In [23]:
n = 1 << logn_top
hhn = n >> 2
power_index = 1

for j in range(0, 1):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [24]:
print(temp)

[0, 0, 4096, 1024, 4096, 1024, 0, 1024, 3072, 2048, 0, 3072, 6144, 0, 3072, 2048, 3072, 0, 0, 1024, 0, 4096, 0, 2048, 0, 2048, 3072, 3072, 5120, 2048, 0, 1024, 2048, 2048, 0, 1024, 2048, 0, 1024, 4096, 0, 1024, 1024, 3072, 0, 1024, 1024, 2048, 4096, 1024, 2048, 0, 4096, 0, 1024, 6144, 0, 4096, 1024, 0, 1024, 2048, 3072, 0, 2048, 0, 3072, 2048, 2048, 1024, 2048, 2048, 3072, 3072, 3072, 0, 0, 3072, 2048, 1024, 1024, 0, 0, 4096, 4096, 0, 4096, 0, 0, 1024, 2048, 2048, 0, 2048, 0, 1024, 0, 2048, 0, 1024, 0, 5120, 4096, 4096, 2048, 7168, 0, 1024, 1024, 4096, 0, 2048, 3072, 1024, 3072, 2048, 2048, 0, 1024, 5120, 0, 0, 2048, 3072, 2048, 2048, 2048, 3072, 2048, 1024, 1024, 0, 1024, 0, 1024, 3072, 1024, 4096, 0, 4096, 3072, 3072, 0, 1024, 3072, 1024, 3072, 5120, 3072, 0, 1024, 1024, 2048, 2048, 1024, 0, 4096, 5120, 4096, 3072, 5120, 1024, 1024, 2048, 1024, 1024, 2048, 3072, 0, 4096, 0, 0, 4096, 1024, 2048, 0, 1024, 3072, 0, 4096, 3072, 0, 2048, 0, 0, 3072, 1024, 6144, 0, 5120, 7168, 0, 2048, 204

In [25]:
print(number_order(temp, P[0]))


2


In [26]:
inv2 = 16775169
inv4 = mod_mul_standard(inv2, inv2, P[0])
mu_inv4 = Mu(inv4, P[0])

inv16 = mod_mul_standard(inv4, inv4, P[0])
mu_inv16 = Mu(inv16, P[0])
inv64 = mod_mul_standard(inv16, inv4, P[0])
mu_inv64 = Mu(inv64, P[0])
inv256 = mod_mul_standard(inv64, inv4, P[0])
mu_inv256 = Mu(inv256, P[0])
inv1024 = mod_mul_standard(inv256, inv4, P[0])
mu_inv1024 = Mu(inv1024, P[0])




for i in range(0, 2*n):
    temp[i] = Barrett_mul(temp[i], inv1024, P[0], mu_inv1024)


print(temp)

check_arrays(temp[0:n], f,P[0])

[0, 0, 4, 1, 4, 1, 0, 1, 3, 2, 0, 3, 6, 0, 3, 2, 3, 0, 0, 1, 0, 4, 0, 2, 0, 2, 3, 3, 5, 2, 0, 1, 2, 2, 0, 1, 2, 0, 1, 4, 0, 1, 1, 3, 0, 1, 1, 2, 4, 1, 2, 0, 4, 0, 1, 6, 0, 4, 1, 0, 1, 2, 3, 0, 2, 0, 3, 2, 2, 1, 2, 2, 3, 3, 3, 0, 0, 3, 2, 1, 1, 0, 0, 4, 4, 0, 4, 0, 0, 1, 2, 2, 0, 2, 0, 1, 0, 2, 0, 1, 0, 5, 4, 4, 2, 7, 0, 1, 1, 4, 0, 2, 3, 1, 3, 2, 2, 0, 1, 5, 0, 0, 2, 3, 2, 2, 2, 3, 2, 1, 1, 0, 1, 0, 1, 3, 1, 4, 0, 4, 3, 3, 0, 1, 3, 1, 3, 5, 3, 0, 1, 1, 2, 2, 1, 0, 4, 5, 4, 3, 5, 1, 1, 2, 1, 1, 2, 3, 0, 4, 0, 0, 4, 1, 2, 0, 1, 3, 0, 4, 3, 0, 2, 0, 0, 3, 1, 6, 0, 5, 7, 0, 2, 2, 1, 0, 2, 0, 5, 1, 4, 0, 0, 1, 1, 2, 1, 3, 4, 5, 1, 0, 0, 1, 4, 2, 0, 0, 2, 4, 3, 3, 3, 5, 1, 0, 0, 0, 1, 0, 2, 2, 0, 0, 0, 1, 0, 2, 3, 1, 0, 0, 4, 1, 3, 0, 1, 0, 1, 4, 0, 1, 2, 1, 2, 3, 0, 3, 5, 4, 4, 1, 2, 1, 1, 1, 1, 5, 5, 1, 2, 5, 1, 4, 1, 0, 0, 2, 6, 0, 4, 4, 0, 0, 0, 7, 4, 4, 1, 0, 0, 0, 0, 0, 1, 4, 2, 4, 1, 6, 2, 9, 2, 2, 3, 2, 1, 0, 0, 2, 2, 0, 1, 4, 3, 3, 3, 1, 1, 0, 2, 0, 0, 0, 4, 1, 7, 0, 0, 1, 2, 3, 4, 

True

In [27]:
Barrett_mul(temp[i], inv1024, P[0], mu_inv1024)

0